批量画图

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import traceback

# 不需要分析的列名列表
EXCLUDE_COLUMNS = [
    'nodes', 'isoform_id', 'identity', 'target', 'uniprotKB_pfam', 
    'uniprotKB_EC_number', 'uniprotKB_clan', 'uniprotKB_EC_class',
    'proteinVec_Filename', 'proteinVec_Pfam', 
    'proteinVec_Gene_Ontology_(biological_process)',
    'proteinVec_Gene_Ontology_(molecular_function)', 
    'proteinVec_Gene_Ontology_(cellular_component)',
    'proteinVec_Gene3D', 'proteinVec_EC_number', 'uniprotKB',
    'proteome_Gene_Name', 'gene_symbol', 'Row.names', 'transcript_id'
]

# 创建输出目录
output_dir = 'check_column_distribution'
os.makedirs(output_dir, exist_ok=True)

# 读取数据
df = pd.read_csv('021.filtered_no_AFDB_sqanti3_classifcation_90928.tsv', sep='\t')

# 检查已经生成的图片
existing_plots = {f.split('_distribution.png')[0] for f in os.listdir(output_dir) if f.endswith('_distribution.png')}
print(f"已经成功生成的图片数量: {len(existing_plots)}")

# 对每一列进行分析
for column in df.columns:
    try:
        # 跳过不需要分析的列
        if column in EXCLUDE_COLUMNS:
            continue
            
        # 跳过已经成功生成的列
        if column in existing_plots:
            print(f"跳过已处理的列: {column}")
            continue
            
        print(f"\n开始处理列: {column}")
        print(f"数据类型: {df[column].dtype}")
        print(f"唯一值数量: {df[column].nunique()}")
        print(f"非空值数量: {df[column].count()}")
        
        # 检查数据大小，如果唯一值太多就跳过
        if df[column].nunique() > 1000:
            print(f"跳过列 {column}: 唯一值数量({df[column].nunique()})太多")
            continue
            
        plt.figure(figsize=(10, 6))
        
        # 判断列的数据类型
        if df[column].dtype in ['object', 'string']:  # 字符串类型
            print("处理类别型数据...")
            # 如果唯一值太多，只显示top N个
            value_counts = df[column].value_counts()
            if len(value_counts) > 20:
                value_counts = value_counts.head(20)
                print(f"警告: {column} 包含超过20个唯一值，只显示前20个")
            
            # 检查数据是否为空
            if len(value_counts) == 0:
                print(f"跳过列 {column}: 没有有效数据")
                plt.close()
                continue
                
            plt.pie(value_counts.values, labels=value_counts.index, autopct='%1.1f%%')
            plt.title(f'Distribution of {column} (Categorical)')
            
        else:  # 数值类型
            print("处理数值型数据...")
            # 移除可能的无限值和空值
            data = df[column].replace([np.inf, -np.inf], np.nan).dropna()
            
            # 检查是否有足够的数据
            if len(data) == 0:
                print(f"跳过列 {column}: 没有有效数值数据")
                plt.close()
                continue
                
            # 检查数据范围
            print(f"数据范围: {data.min()} to {data.max()}")
            
            # 绘制分布图
            sns.histplot(data=data, bins=30, kde=True)
            plt.title(f'Distribution of {column} (Numerical)')
            plt.xlabel(column)
            plt.ylabel('Count')
        
        # 保存图片前先检查图形大小
        fig = plt.gcf()
        fig_size = fig.get_size_inches()
        dpi = fig.get_dpi()
        pixel_size = (fig_size[0] * dpi, fig_size[1] * dpi)
        
        # 如果图片尺寸太大就跳过
        if pixel_size[0] > 65000 or pixel_size[1] > 65000:
            print(f"跳过列 {column}: 图片尺寸太大 ({pixel_size[0]}x{pixel_size[1]} pixels)")
            plt.close()
            continue
        
        # 保存图片
        plt.tight_layout()
        save_path = os.path.join(output_dir, f'{column}_distribution.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"成功保存图片: {save_path}")
        
    except Exception as e:
        print(f"\n处理列 {column} 时出错:")
        print(f"错误类型: {type(e).__name__}")
        print(f"错误信息: {str(e)}")
        plt.close()  # 确保关闭当前图形
        continue  # 继续处理下一列

print("\n处理完成！")

已经成功生成的图片数量: 0

开始处理列: chrom
数据类型: object
唯一值数量: 25
非空值数量: 90929
处理类别型数据...
警告: chrom 包含超过20个唯一值，只显示前20个
成功保存图片: check_column_distribution/chrom_distribution.png

开始处理列: strand
数据类型: object
唯一值数量: 2
非空值数量: 90929
处理类别型数据...
成功保存图片: check_column_distribution/strand_distribution.png

开始处理列: length
数据类型: float64
唯一值数量: 8093
非空值数量: 90929
跳过列 length: 唯一值数量(8093)太多

开始处理列: exons
数据类型: float64
唯一值数量: 74
非空值数量: 90929
处理数值型数据...
数据范围: 1.0 to 78.0
成功保存图片: check_column_distribution/exons_distribution.png

开始处理列: structural_category
数据类型: object
唯一值数量: 8
非空值数量: 90929
处理类别型数据...
成功保存图片: check_column_distribution/structural_category_distribution.png

开始处理列: associated_gene
数据类型: object
唯一值数量: 14384
非空值数量: 90929
跳过列 associated_gene: 唯一值数量(14384)太多

开始处理列: associated_transcript
数据类型: object
唯一值数量: 25996
非空值数量: 90929
跳过列 associated_transcript: 唯一值数量(25996)太多

开始处理列: ref_length
数据类型: float64
唯一值数量: 8110
非空值数量: 90920
跳过列 ref_length: 唯一值数量(8110)太多

开始处理列: ref_exons
数据类型: float64
唯一值数量: 104
非空值数量: 90920
处理数